# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql ('SELECT * FROM InsertTableName', engine)
X = df ['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    pass
#converting all the text to lowercase
    text.lower()
    # removing punctuation characters from the text and replacing them with an empty space
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # use the word tokenizer to convert text into tokens
    tokens = word_tokenize(text)
    # removing stopwords by calling a "for loop" for the tokens and using english stopwords from nltk!
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    # Using lemmatization to strip all the words
    tokens = [WordNetLemmatizer().lemmatize(word).strip() for word in tokens]
    # finally return the tokens
    return tokens

print(X[3])
print(tokenize(X[3]))

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['UN', 'report', 'Leogane', '80', '90', 'destroyed', 'Only', 'Hospital', 'St', 'Croix', 'functioning', 'Needs', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
 # train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pred = pipeline.predict(X_test)

In [8]:
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)

related
             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1500
          1       0.85      0.91      0.88      4993
          2       0.32      0.39      0.35        61

avg / total       0.80      0.81      0.80      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5432
          1       0.80      0.42      0.55      1122

avg / total       0.87      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6533
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.74      0.85      0.79      3801
          1       0.74      0.59      0.66      2753

avg / total       0.74      0.74      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params
parameters = {
        'vect__max_df':[0.85,1.0],
        'clf__estimator__n_estimators': [22, 50]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, n_jobs=4)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=22, vect__max_df=0.85 ..............
[CV] clf__estimator__n_estimators=22, vect__max_df=0.85 ..............
[CV] clf__estimator__n_estimators=22, vect__max_df=0.85 ..............
[CV] clf__estimator__n_estimators=22, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=22, vect__max_df=0.85, score=0.251754653646628, total=13.2min
[CV] clf__estimator__n_estimators=22, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=22, vect__max_df=1.0, score=0.23985352456515105, total=13.2min
[CV] clf__estimator__n_estimators=22, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=22, vect__max_df=0.85, score=0.23985352456515105, total=13.2min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.85 ..............
[CV]  clf__estimator__n_estimators=22, vect__max_df=0.85, score=0.24961855355508086, total=13.2min
[CV] clf__estimator__n_estimators=50, vect__max_df

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 36.4min


[CV]  clf__estimator__n_estimators=22, vect__max_df=1.0, score=0.25480622520598106, total=13.2min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.85, score=0.26167226121452547, total=21.4min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............


[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 45.5min remaining: 32.5min


[CV]  clf__estimator__n_estimators=50, vect__max_df=0.85, score=0.25205981080256334, total=21.5min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.85, score=0.25358559658223984, total=21.7min


[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 64.0min remaining: 21.3min


[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.25282270369240156, total=21.7min
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.2638083613060726, total=20.1min
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.2625877326823314, total=20.1min


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 68.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__max_df': [0.85, 1.0], 'clf__estimator__n_estimators': [22, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = cv.predict(X_test)
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)

In [ ]:
model = cv

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
Y_pred_tuned = model.predict (X_test)
#converting to a dataframe
Y_pred_tuned = pd.DataFrame (Y_pred_tuned, columns = Y_test.columns)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
#pickle.dump(model, open('classifier.pkl', 'wb'))
pkl_filename = 'classifier.pkl'
with open(pkl_filename, 'wb') as file:  
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.